### Download Glove Embeddings

In [ ]:
import os
import imp

GLOVE_PATH = os.path.join("glove", "glove.840B.300d.txt")

if not os.path.exists(GLOVE_PATH):
    !wget "https://nlp.stanford.edu/data/glove.840B.300d.zip"
    !unzip "glove.840B.300d.zip" -d "glove"
    !rm "glove.840B.300d.zip"

try:
    imp.find_module("senteval")
    senteval_exists = True
except:
    senteval_exists = False

if not senteval_exists:
    !git clone https://github.com/facebookresearch/SentEval.git
    %cd SentEval
    !python setup.py install
    %cd data/downstream/
    !bash get_transfer_data.bash
    %cd ../../..

In [1]:
import torch
from dataset.dataset_utils import get_glove_vocab_index_mapping
from dataset.dataset_utils import get_glove_embeddings
from models.utils import get_encoder
import warnings
import nltk
import os
nltk.download("punkt", quiet=True)
warnings.filterwarnings("ignore")

# change this variable to reflect the path you downloaded senteval to
PATH_TO_SENTEVAL_DATA = f"{os.environ['HOME']}/sentence_embeddings_learning/SentEval/data" 


GLOVE_PATH = os.path.join("glove", "glove.840B.300d.txt")
glove_mapping = get_glove_vocab_index_mapping(GLOVE_PATH)
device = "cpu"

def process_inputs(premise, hypothesis):
    premise = nltk.word_tokenize(premise.lower())
    hypothesis = nltk.word_tokenize(hypothesis.lower())

    max_sentence_length = max(len(premise), len(hypothesis))

    premise = [
        glove_mapping[token]
        if token in glove_mapping else 0
        for token in premise
    ]

    hypothesis = [
        glove_mapping[token]
        if token in glove_mapping else 0
        for token in hypothesis
    ]

    premise = torch.tensor([
        premise + [1] * (max_sentence_length - len(premise))
    ])

    hypothesis = torch.tensor([
        hypothesis + [1] * (max_sentence_length - len(hypothesis))
    ])

    return premise, hypothesis


def inference(encoder, classifier, premise, hypothesis):
    u = encoder(premise)
    v = encoder(hypothesis)
    encoder_output = torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

    preds = classifier(encoder_output)
    preds = torch.argmax(preds)
    preds = preds.item()

    if preds == 1:
        return "neutral"
    elif preds == 0:
        return "entailment"
    else:
        return "contradiction"


In [2]:
ENCODER_PATH = os.path.join("saved_models", "MaxPoolingBiLSTM.pt")
CLASSIFIER_PATH = os.path.join("saved_models", "MaxPoolingBiLSTM_classifier.pt")

premise = "People are conversing at a dining table under a canopy."
hypothesis = "People are screaming at a boxing match."

encoder = torch.load(ENCODER_PATH)
classifier = torch.load(CLASSIFIER_PATH)

encoder.to(device)
classifier.to(device)

premise, hypothesis = process_inputs(premise, hypothesis)

print("Entailment of the provided premise and hypothesis is: ", inference(encoder, classifier, premise, hypothesis))

Entailment of the provided premise and hypothesis is:  contradiction


### Baseline encoder results

In [3]:
ENCODER_PATH = os.path.join("saved_models", "BaselineEncoder.pt")
CLASSIFIER_PATH = os.path.join("saved_models", "BaselineEncoder_classifier.pt")

!python evaluate.py --encoder_checkpoint_path {ENCODER_PATH} --classifier_checkpoint_path {CLASSIFIER_PATH} --sent_eval_data_path {PATH_TO_SENTEVAL_DATA}

2023-04-22 15:23:17,860 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 15:23:18,004 : https://huggingface.co:443 "GET /api/datasets/snli HTTP/1.1" 200 2469
2023-04-22 15:23:18,006 : Starting new HTTPS connection (1): s3.amazonaws.com:443
2023-04-22 15:23:18,422 : https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/snli/snli.py HTTP/1.1" 200 0
2023-04-22 15:23:18,430 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 15:23:18,539 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/snli.py HTTP/1.1" 200 0
2023-04-22 15:23:18,545 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 15:23:18,652 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/dataset_infos.json HTTP/1.1" 200 0
2023-04-22 15:23:18,658 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 15:23:18,779 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/README.md HTTP/1.1" 200 0
2023-04-22 15:23:18,799

### One direction LSTM encoder results

In [4]:
ENCODER_PATH = os.path.join("saved_models", "BaselineLSTM.pt")
CLASSIFIER_PATH = os.path.join("saved_models", "BaselineLSTM_classifier.pt")

!python evaluate.py --encoder_checkpoint_path {ENCODER_PATH} --classifier_checkpoint_path {CLASSIFIER_PATH} --sent_eval_data_path {PATH_TO_SENTEVAL_DATA}

2023-04-22 15:29:40,470 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 15:29:40,628 : https://huggingface.co:443 "GET /api/datasets/snli HTTP/1.1" 200 2469
2023-04-22 15:29:40,631 : Starting new HTTPS connection (1): s3.amazonaws.com:443
2023-04-22 15:29:41,030 : https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/snli/snli.py HTTP/1.1" 200 0
2023-04-22 15:29:41,037 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 15:29:41,147 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/snli.py HTTP/1.1" 200 0
2023-04-22 15:29:41,151 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 15:29:41,260 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/dataset_infos.json HTTP/1.1" 200 0
2023-04-22 15:29:41,267 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 15:29:41,392 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/README.md HTTP/1.1" 200 0
2023-04-22 15:29:41,414

### Bidirectional LSTM encoder results

In [5]:
ENCODER_PATH = os.path.join("saved_models", "BiLSTM.pt")
CLASSIFIER_PATH = os.path.join("saved_models", "BiLSTM_classifier.pt")

!python evaluate.py --encoder_checkpoint_path {ENCODER_PATH} --classifier_checkpoint_path {CLASSIFIER_PATH} --sent_eval_data_path {PATH_TO_SENTEVAL_DATA}

2023-04-22 18:05:11,989 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 18:05:12,128 : https://huggingface.co:443 "GET /api/datasets/snli HTTP/1.1" 200 2469
2023-04-22 18:05:12,130 : Starting new HTTPS connection (1): s3.amazonaws.com:443
2023-04-22 18:05:12,541 : https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/snli/snli.py HTTP/1.1" 200 0
2023-04-22 18:05:12,557 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 18:05:12,666 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/snli.py HTTP/1.1" 200 0
2023-04-22 18:05:12,678 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 18:05:12,792 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/dataset_infos.json HTTP/1.1" 200 0
2023-04-22 18:05:12,805 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-22 18:05:12,920 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/README.md HTTP/1.1" 200 0
2023-04-22 18:05:12,935

### Max pooling Bidirectional LSTM encoder results

In [6]:
ENCODER_PATH = os.path.join("saved_models", "MaxPoolingBiLSTM.pt")
CLASSIFIER_PATH = os.path.join("saved_models", "MaxPoolingBiLSTM_classifier.pt")

!python evaluate.py --encoder_checkpoint_path {ENCODER_PATH} --classifier_checkpoint_path {CLASSIFIER_PATH} --sent_eval_data_path {PATH_TO_SENTEVAL_DATA}

2023-04-23 00:09:01,588 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-23 00:09:01,730 : https://huggingface.co:443 "GET /api/datasets/snli HTTP/1.1" 200 2469
2023-04-23 00:09:01,732 : Starting new HTTPS connection (1): s3.amazonaws.com:443
2023-04-23 00:09:02,145 : https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/snli/snli.py HTTP/1.1" 200 0
2023-04-23 00:09:02,162 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-23 00:09:02,267 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/snli.py HTTP/1.1" 200 0
2023-04-23 00:09:02,278 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-23 00:09:02,398 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/dataset_infos.json HTTP/1.1" 200 0
2023-04-23 00:09:02,408 : Starting new HTTPS connection (1): huggingface.co:443
2023-04-23 00:09:02,522 : https://huggingface.co:443 "HEAD /datasets/snli/resolve/main/README.md HTTP/1.1" 200 0
2023-04-23 00:09:02,540

### Error analysis

In this project, 4 models have been trained namely a baseline encoder that averages Glove embeddings to produce a sentence representation, 2 LSTM-based models that output the last hidden state as the sentence embedding, and a LSTM-based model that does max pooling over the hidden states outputted by the LSTM layer. The performance ranking of these models is the following in descending order from best to worst:

1) Max Pooling BiLSTM
2) BiLSTM last hidden state
3) LSTM last hidden state
4) Glove embeddings average

The Baseline encoder, the one that just averages word embeddings and does not learn anything is expected to perform worst since it does not learn specialized weights for the SNLI dataset. The LSTM models that use the last hidden state of the LSTM layer as the sentence representation should perform better than the baseline because we actually learn weights specific to the given dataset and it incorporates information from all the tokens in the sentence. It is worth mentioning here that the BiLSTM should be better than the Unidirectional LSTM because when aggregating information about the sentence into the last hidden state of the LSTM layer, most of the information about the tokens at the beginning of the sentence is going to be mostly lost because of the forgetting mechanism incorporated into the LSTM layers. This behavior is exacerbated for longer sentences, so using sentence information from both directions barely solves this issue. Last but not least, the Max Pooling BiLSTM should perform the best out of these models because it actually uses the most useful features from each token from both directions according to the LSTM layer.